In [1]:
import pandas as pd
import pickle
from helper_functions import Helper
from helper_functions import Coinmetrics

In [2]:
if __name__ == "__main__":
    # set args
    TARGET_METRICS = ['AdrActCnt', 'AdrActContCnt', 'AdrActRecCnt', 'AdrActSentCnt', 'AdrBalCnt',
        'AdrBalUSD100Cnt', 'AdrBalUSD100KCnt', 'AdrBalUSD10Cnt', 'AdrBalUSD10KCnt', 'AdrBalUSD10MCnt',
        'AdrBalUSD1Cnt', 'AdrBalUSD1KCnt', 'AdrBalUSD1MCnt', 'CapAct1yrUSD',
        'CapFutExp10yrUSD', 'CapMVRVCur', 'CapMVRVFF', 'CapMrktCurUSD', 'CapMrktEstUSD', 'CapMrktFFUSD',
        'CapRealUSD', 'ContERC1155Cnt', 'ContERC20Cnt', 'ContERC721Cnt',
        'DiffMean', 'FeeMeanUSD', 'FeeMedUSD', 'FeeRevPct', 'FeeTotUSD', 'FlowInExInclUSD', 'FlowInExUSD',
        'FlowMinerIn0HopAllUSD', 'FlowMinerIn1HopAllUSD', 'FlowMinerNet0HopAllUSD', 'FlowMinerNet1HopAllUSD',
        'FlowMinerOut0HopAllUSD', 'FlowMinerOut1HopAllUSD', 'FlowOutExInclUSD', 'FlowOutExUSD', 'FlowTfrFromExCnt',
        'FlowTfrFromExInclCnt', 'FlowTfrToExCnt', 'FlowTfrToExInclCnt', 'FlowToCLCont', 'GasUsedTx', 'GasUsedTxMean',
        'HashRate', 'IssContNtv', 'IssContPctAnn', 'IssContPctDay', 'IssContUSD', 'IssTotUSD', 'MCRC', 'MCTC',
        'MOMR', 'MRI0HopAll30d', 'MRI1HopAll30d', 'NDF', 'NVTAdj', 'NVTAdjFF', 'PriceUSD', 'PuellMulCont', 'PuellMulRev',
        'PuellMulTot', 'RCTC', 'RVTAdj', 'ReferenceRateUSD', 'RevAllTimeUSD', 'RevHash1yAvgNtv', 'RevHash1yAvgUSD',
        'RevHashUSD', 'RevUSD', 'SER', 'SOPR', 'SOPROut', 'SenderCntCLCont', 'SenderTotCLCont', 'SplyAct10yr',
        'SplyAct180d', 'SplyAct1d', 'SplyAct1yr', 'SplyAct2yr', 'SplyAct30d', 'SplyAct3yr', 'SplyAct4yr', 'SplyAct5yr',
        'SplyAct7d', 'SplyAct90d', 'SplyActEver', 'SplyActPct1yr', 'SplyAdrBalUSD1', 'SplyAdrBalUSD10',
        'SplyAdrBalUSD100', 'SplyAdrBalUSD100K', 'SplyAdrBalUSD10K', 'SplyAdrBalUSD10M', 'SplyAdrBalUSD1K',
        'SplyAdrBalUSD1M', 'SplyAdrTop100', 'SplyAdrTop10Pct', 'SplyAdrTop1Pct', 'SplyBurntUSD', 'SplyContUSD',
        'SplyCur', 'SplyExUSD', 'SplyExpFut10yr', 'SplyFF', 'SplyMiner0HopAllUSD', 'SplyMiner1HopAllUSD',
        'SplyRvv180d', 'SplyRvv1yr', 'SplyRvv2yr', 'SplyRvv30d', 'SplyRvv3yr', 'SplyRvv4yr', 'SplyRvv5yr', 
        'SplyRvv7d', 'SplyRvv90d', 'SplyUTXOLoss', 'SplyUTXOProf', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD',    
        'TxTfrValContCallUSD', 'TxTfrValDayDst', 'TxTfrValDayDstMean', 'TxTfrValMeanUSD', 'TxTfrValMedUSD',
        'TxTfrValNtv', 'TxTfrValRecNtv', 'TxTfrValSentNtv', 'TxTfrValUSD', 'UTXOAgeMean', 'UTXOAgeMed', 
        'UTXOAgeValMean', 'UTXOCnt', 'UTXODay', 'UTXOLossCnt', 'UTXOLossUnrealUSD', 'UTXOProfCnt',
        'UTXOProfUnrealUSD', 'VelAct1yr', 'VelCur1yr', 'WalActCnt', 'WalActRecCnt', 'WalActSentCnt', 'WalBalCnt',
        'mempool_count', 'mempool_fee', 'mempool_fee_mean', 'mempool_fee_median', 'mempool_feerate_mean',  
        'mempool_feerate_median', 'mempool_output_value', 'mempool_size', 'mempool_vsize', 
        'mining_reward_mean', 'mining_reward_spread']
    CW_IN_FP = '../data/derived/cm_to_coinapi_cw.pkl'
    ASSET_IN_FP = '../data/clean/asset_universe_dict.pickle'
    CM_API_FP = '../../admin/coinmetrics.txt'
    BASE_URL = 'https://api.coinmetrics.io/v4/'
    PANEL_OUT_FP = '../data/raw/coinmetrics_panel_hourly.pkl'
    TARGET_US_EXCHANGES = ['binance.us',  'bitstamp', 'coinbase', 'crypto.com', 'ftx.us', 
        'gemini', 'kraken', 'kucoin']
    TARGET_FREQ = '1h'
    STUDY_START = '2016-07-01'
    STUDY_END = '2023-01-03'
    
    # import api key
    with open(CM_API_FP) as f:
        API_KEY = f.readlines()
        API_KEY = API_KEY[0].strip()
    BASE_PARAMS = {'api_key': API_KEY}

    # Import asset universe and cw
    cw_df = pd.read_pickle(CW_IN_FP)
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe = Helper.findUniqueAssets(asset_universe_dict)

    # Pull info on assets
    assets_df = Coinmetrics.pullAssetInfo(BASE_URL, BASE_PARAMS, asset_universe+['usdt', 'usdc'])

    # pull meta data on target exchanges
    exchanges_df = Coinmetrics.pullExchangeInfo(BASE_URL, BASE_PARAMS, TARGET_US_EXCHANGES)

    # pull meta data markets and subset down to target markets
    markets_df = Coinmetrics.pullAndFormRelevantMarkets(exchanges_df, assets_df, BASE_URL, BASE_PARAMS, asset_universe)

    # pull reference rates for USDC and USDT to use for exchange rate calculations
    usd_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END, 
                                          TARGET_FREQ, ['usdc', 'usdt'], ['ReferenceRateUSD'])
    usd_df = Coinmetrics.cleanUSDPrices(usd_df, TARGET_FREQ)

    # pull prices and volumes for all markets of interest
    ohlcv_df = Coinmetrics.pullOHLCV(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, TARGET_FREQ, markets_df[:10])
    ohlcv_df = Coinmetrics.cleanOHLCV(ohlcv_df, TARGET_FREQ, markets_df, usd_df)
    # TODO DROP THE MARKETS_DF SUBSET ABOVE

    # pull asset metrics
    panel_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END,
                                            TARGET_FREQ, asset_universe[:3], TARGET_METRICS[:10])
    panel_df = Coinmetrics.cleanPanel(panel_df)
    # TODO DROP THE ASSET UNIVERSE AND TARGET METRICS SUBSETS ABOVE
    panel_df = panel_df.merge(ohlcv_df, on=['date', 'asset'], how='outer', validate='one_to_one')
    


Processing the 1th asset (50.00%): usdc
Processing the 2th asset (100.00%): usdt
Processing market #1 (10.00%): binance.us-1inch-usd-spot
Processing market #2 (20.00%): binance.us-1inch-usdt-spot
Processing market #3 (30.00%): binance.us-aave-usd-spot
Processing market #4 (40.00%): binance.us-aave-usdt-spot
Processing market #5 (50.00%): binance.us-ach-usd-spot
Processing market #6 (60.00%): binance.us-ach-usdt-spot
Processing market #7 (70.00%): binance.us-ada-usd-spot
Processing market #8 (80.00%): binance.us-ada-usdc-spot
Processing market #9 (90.00%): binance.us-ada-usdt-spot
Processing market #10 (100.00%): binance.us-algo-usd-spot


The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 1.33 seconds.


Processing the 1th asset (33.33%): 1inch


The API call failed with error: 403 Client Error: Forbidden for url: https://api.coinmetrics.io/v4/timeseries/asset-metrics?api_key=hMkOUrQIV5EvIGiRyxZ9&page_size=10000&limit_per_asset=10000&end_inclusive=False&assets=1inch&metrics=AdrNewBalCnt&frequency=1d&start_time=2020-12-23&end_time=2021-01-01
403 Forbidden Error - Will not retry
The API call failed with error: 403 Client Error: Forbidden for url: https://api.coinmetrics.io/v4/timeseries/asset-metrics?api_key=hMkOUrQIV5EvIGiRyxZ9&page_size=10000&limit_per_asset=10000&end_inclusive=False&assets=1inch&metrics=AdrNewBalCnt&frequency=1d&start_time=2021-01-01&end_time=2021-08-18
403 Forbidden Error - Will not retry


for 1inch AdrNewBalCnt, we have no data.


...

for aave ContBalCnt, we have no data.


The API call failed with error: 403 Client Error: Forbidden for url: https://api.coinmetrics.io/v4/timeseries/asset-metrics?api_key=hMkOUrQIV5EvIGiRyxZ9&page_size=10000&limit_per_asset=10000&end_inclusive=False&assets=aave&metrics=ContCnt&frequency=1d&start_time=2020-10-02&end_time=2021-01-01
403 Forbidden Error - Will not retry
The API call failed with error: 403 Client Error: Forbidden for url: https://api.coinmetrics.io/v4/timeseries/asset-metrics?api_key=hMkOUrQIV5EvIGiRyxZ9&page_size=10000&limit_per_asset=10000&end_inclusive=False&assets=aave&metrics=ContCnt&frequency=1d&start_time=2021-01-01&end_time=2021-03-02
403 Forbidden Error - Will not retry


for aave ContCnt, we have no data.


The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 0.65 seconds.
The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 0.81 seconds.


Processing the 3th asset (100.00%): aca


In [3]:
assets_df.head(1)

,asset,full_name,exchanges,markets,metrics
0,1inch,1inch,"[bibox, binance, binance.us, bitfinex, bitstam...","[bibox-1inch-usdt-spot, binance-1INCHUSDT-futu...","[{'metric': 'AdrAct30dCnt', 'frequencies': [{'..."


In [4]:
exchanges_df.head(1)

,exchange,markets,min_time,max_time,metrics
0,binance.us,"[binance.us-123-456-spot, binance.us-1inch-usd...",2019-09-04T13:06:51.554000000Z,2023-03-20T22:16:51.425000000Z,"[{'metric': 'volume_reported_spot_usd_1d', 'fr..."


In [5]:
markets_df.head(1)

,market,exchange,base,quote,min_time,max_time
0,binance.us-1inch-usd-spot,binance.us,1inch,usd,2021-12-10T01:00:00.000000000Z,2023-03-20T22:16:10.000000000Z


In [6]:
usd_df.head(1)

,date,usd_per_usdc,usd_per_usdt
0,2016-07-01,NaN,1.00326


In [7]:
ohlcv_df.head(1)

,date,asset,usd_per_token_cm,usd_volume_cm,trades_cm
0,2021-08-20,aave,411.115173,443917.96875,944.0


In [ ]:
# TODO move functions to the helper function call

# TODO adjust the OG script to see if they work for it too

In [ ]:
# TODO FOR TARGET EXCHANGES AS WELL AS A FEW OTHER MAJORS pull all the available exchange metrics for my study period
# -scope supported metrics for all my markets to see if anythign else interesting to pull
# -pull metrics from exchanges_df as well as start adn end dates to pull all teh data
# -try to get inflows and outflows?
# -pull open interest in futures? maybe as macro variable but lets see if i can get for a bunch of assets
# -scope the endpoint below to see if there are more exchange metrics that i am missing
# curl "https://api.coinmetrics.io/v4/catalog/exchange-metrics?pretty=true&api_key=<your_key>"
# MAYBE ALSO JUST GET THREE OTHER MAJOR EXCHAGNES

# TODO: pull defi balance sheets

# TODO: taxonomy?

In [ ]:
# TODO PUlL BEST BID AND ASK FOR ALL THOSE MARKETS AT HOURLY FREQ 
# - "timeseries/market-quotes"
# -problem is can't specify frequency

base_url = BASE_URL
base_params = BASE_PARAMS

target_url = "timeseries/market-quotes"
url = f"{base_url}{target_url}"

params = base_params.copy()
params['start_time'] = '2021-09-30'
params['end_time'] = '2021-10-30'
params['markets'] = 'coinbase-btc-usd-spot'
params['page_size'] = 10000
params['end_inclusive'] = False

response_json = Helper.makeApiCall(url, headers={}, params=params)

df = pd.DataFrame(response_json['data'])

# TODO THIS IS GOING TO TAKE 3-4 DAYS TO RUN